In [1]:
using Gridap
using Gridap.Fields
using Gridap.Geometry
using GridapGmsh

using ForwardDiff

# Geometry

In [2]:
# Enclosure dimensions (enclosure is not centered, the core is)
hencl1 = 65.5e-2;   # Height above the x-axis
hencl2 = -53.5e-2;  # Height below the x-axis
wencl  = 104e-2;    # Width

# Core dimensions
wcore = 84e-2;
hcore = 100e-2;

# Core gap dimensions (left and right are identical)
wgap = 17e-2;
hgap = 76e-2;
mgap = 17e-2;

# HV winding dimensions (all phases left/right are identical)
wwhv = 3e-2;
hwhv = 74e-2;
mwhv = 14.75e-2;
Awhv = wwhv * hwhv;

# LV winding dimensions (all phases left/right are identical)
wwlv = 2e-2;
hwlv = 74e-2;
mwlv = 11.25e-2;
Awlv = wwlv * hwlv;

# Mesh densities
lc1 = 2e-2;      # Enclosure & core outer
lc2 = 1e-2;      # Core inner
lc3 = 1e-2;      # HV windings
lc4 = 1e-2;      # LV windings

# Gridap Setup

In [3]:
# Define norm of a complex vector for Gridap to use on the CellField which results from Bh = ∇(uh)
norm_complex(a::VectorValue{2, ComplexF64}) = sqrt(norm(a[1])^2 + norm(a[2])^2)
norm_complex(a::VectorValue{2, Float64}) = norm(a)
norm_complex(a::CellField) = Operation(norm_complex)(a)

norm_complex (generic function with 3 methods)

# Linear BH Curve

In [ ]:
# Load mesh
model = GmshDiscreteModel("geo/transformer_stedin.msh")
Ω = Triangulation(model)

# Define Lagrangian reference element
order = 2;
reffe = ReferenceFE(lagrangian, Float64, order)
dΩ = Measure(Ω, 2*order)

# Construct Lagrangian test space with dirichlet condition on the right boundary node
V = TestFESpace(model, reffe, conformity = :H1, dirichlet_tags = ["Enclosure"], vector_type = Vector{ComplexF64})
U = TrialFESpace(V, [0])

In [ ]:
Ip = 0;       # Primary peak phase current
Is = 777.62;  # Secondary peak phase current
Np = 266;
Ns = 6;

omega = 2*pi*50;  # Frequency

mu0 = 4e-7 * pi;

# Calculate current density in the windings
Jp = Np * Ip / Awhv;
Js = Ns * Is / Awlv;

In [ ]:
labels = get_face_labeling(model)
dimension = num_cell_dims(model)
tags = get_face_tag(labels, dimension)

# Construct a cell field consisting of the physical domain tags
τ = CellField(tags, Ω)

const tag_core = get_tag_from_name(labels, "Core");

const tag_lv1l = 10; const tag_lv1r = 11;
const tag_lv2l = 12; const tag_lv2r = 13;
const tag_lv3l = 14; const tag_lv3r = 15;

# Source current density
function fsource(tag)
    if tag == tag_lv1l        # LV winding phase 1 left
        return Js * exp(1im * 2pi/3);
    elseif tag == tag_lv1r    # LV winding phase 1 right
        return -Js * exp(1im * 2pi/3);
    elseif tag == tag_lv2l    # LV winding phase 2 left
        return Js;
    elseif tag == tag_lv2r    # LV winding phase 2 right
        return -Js;
    elseif tag == tag_lv3l    # LV winding phase 3 left
        return Js * exp(-1im * 2pi/3);
    elseif tag == tag_lv3r    # LV winding phase 3 right
        return -Js * exp(-1im * 2pi/3);
    else
        return 0.0 + 0.0im;
    end
end

# Permeability model
function fmur(tag)
    if tag == tag_core
        return 1000.0;
    else
        return 1.0;
    end
end

function fnu(tag)
    return 1 / (mu0 * fmur(tag));
end

# Conductivity
function fsigma(tag)
    if tag == tag_core
        return 0.1;
    else
        return 0.0;
    end
end

In [ ]:
# Define the weak form (bilinear and linear terms)
a(u,v) = ∫( (fnu ∘ τ) ⋅ ∇(u) ⋅ ∇(v) )dΩ + ∫( 1im ⋅ omega ⋅ (fsigma ∘ τ) ⋅ u ⋅ v )dΩ
b(v)   = ∫( (fsource ∘ τ) ⋅ v )dΩ;
op = AffineFEOperator(a, b, U, V)

# Solve the linear FE system with LU solver
ls = LUSolver()
solver = LinearFESolver(ls)
uh = solve(solver, op);

In [ ]:
# Post-processing for magnetic field and current density
Bh = norm_complex(gradient(uh));

J0    = (fsource ∘ τ); # Source current density
Jeddy = 1im ⋅ omega ⋅ (fsigma ∘ τ) ⋅ mean(uh); # Eddy current density

In [ ]:
writevtk(Ω, "images/transformer_gridap/transformer1", cellfields=["Az"=>abs(uh), "imAz"=>imag(uh), "normB" => Bh, "normJ" => abs(J0 + Jeddy)])

![Result: Magnetic Flux Density](images/transformer_gridap/transformer1.png)

# Non-linear BH Curve

In [4]:
# Load mesh
model = GmshDiscreteModel("geo/transformer_stedin.msh")
Ω = Triangulation(model)

# Define Lagrangian reference element
order = 2;
reffe = ReferenceFE(lagrangian, Float64, order)
dΩ = Measure(Ω, 2*order)

# Construct Lagrangian test space with dirichlet condition on the right boundary node
V = TestFESpace(model, reffe, conformity = :H1, dirichlet_tags = ["Enclosure"], vector_type = Vector{ComplexF64})
U = TrialFESpace(V, [0])

Info    : Reading 'geo/transformer_stedin.msh'...
Info    : 168 entities
Info    : 10385 nodes
Info    : 20768 elements
Info    : Done reading 'geo/transformer_stedin.msh'


TrialFESpace()

In [5]:
Ip = 0;       # Primary peak phase current
Is = 777.62;  # Secondary peak phase current
Np = 266;
Ns = 6;

omega = 2*pi*50;  # Frequency

mu0 = 4e-7 * pi;

# Calculate current density in the windings
Jp = Np * Ip / Awhv;
Js = Ns * Is / Awlv;

In [6]:
labels = get_face_labeling(model)
dimension = num_cell_dims(model)
tags = get_face_tag(labels, dimension)

# Construct a cell field consisting of the physical domain tags
τ = CellField(tags, Ω)

const tag_core = get_tag_from_name(labels, "Core");

const tag_lv1l = 10; const tag_lv1r = 11;
const tag_lv2l = 12; const tag_lv2r = 13;
const tag_lv3l = 14; const tag_lv3r = 15;

# Source current density
function fsource(tag)
    if tag == tag_lv1l        # LV winding phase 1 left
        return Js * exp(1im * 2pi/3);
    elseif tag == tag_lv1r    # LV winding phase 1 right
        return -Js * exp(1im * 2pi/3);
    elseif tag == tag_lv2l    # LV winding phase 2 left
        return Js;
    elseif tag == tag_lv2r    # LV winding phase 2 right
        return -Js;
    elseif tag == tag_lv3l    # LV winding phase 3 left
        return Js * exp(-1im * 2pi/3);
    elseif tag == tag_lv3r    # LV winding phase 3 right
        return -Js * exp(-1im * 2pi/3);
    else
        return 0.0 + 0.0im;
    end
end

# Permeability model
bh_a = 2.12e-4; 
bh_b = 7.358;
bh_c = 1.18e7;
mu0  = 4e-7 * pi;
fmur_core(B) = 1 / (bh_a + (1 - bh_a) * B^(2*bh_b) / (B^(2*bh_b) + bh_c));
fdmur_core(B) = ForwardDiff.derivative(fmur_core, B);

function fmur(tag, ∇u)
    if tag == tag_core
        normB = norm_complex(∇u);
        return fmur_core(normB);
    else
        return 1.0;
    end
end

function fdmur(tag, ∇du, ∇u)
    if tag == tag_core
        normB = norm_complex(∇u);
        return fdmur_core(normB) ⋅ (∇du ⊙ ∇u);
    else
        return 0.0 + 0.0im;
    end
end

function fnu(tag, ∇u)
    return 1 / (mu0 * fmur(tag, ∇u));
end

function fdnu(tag, ∇du, ∇u)
    return -1 / (mu0 * fmur(tag, ∇u)^2) * fdmur(tag, ∇du, ∇u);
end

# Conductivity
function fsigma(tag)
    if tag == tag_core
        return 0.0;
    else
        return 0.0;
    end
end

fsigma (generic function with 1 method)

In [9]:
# Define the weak form (bilinear and linear terms)
res(u,v) = ∫( (fnu ∘ (τ, ∇(u))) ⋅ ∇(u) ⋅ ∇(v) )dΩ - ∫( (fsource ∘ τ) ⋅ v )dΩ; # + ∫( 1im ⋅ omega ⋅ (fsigma ∘ τ) ⋅ u ⋅ v )dΩ;
jac(u, du, v) = ∫( (fdnu ∘ (τ, ∇(du), ∇(u))) ⋅ ∇(u) ⋅ ∇(v) )dΩ + ∫( (fnu ∘ (τ, ∇(u))) ⋅ ∇(du) ⋅ ∇(v) )dΩ; # + ∫( 1im ⋅ omega ⋅ (fsigma ∘ τ) ⋅ du ⋅ v )dΩ;
op = FEOperator(res, jac, U, V)

ls = BackslashSolver()
nls = NLSolver(ls, method = :trust_region, show_trace = true)
solver = FESolver(nls)

x   = zeros(Complex{Float64}, num_free_dofs(U))
uh0 = FEFunction(U, x)
uh, = solve!(uh0, solver, op)

Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     1.146565e+01              NaN
     1     1.145763e+01     2.909556e-07
     2     1.144154e+01     5.818369e-07
     3     1.140915e+01     1.163408e-06
     4     1.134348e+01     2.325972e-06
     5     1.120841e+01     4.649993e-06
     6     1.092173e+01     9.300442e-06
     7     1.026521e+01     1.881520e-05
     8     1.003032e+01     6.195531e-05
     9     9.199221e+00     2.438788e-04
    10     9.941258e+00     5.805903e-04
    11     9.029408e+00     4.981106e-04
    12     9.468732e+00     7.555268e-04
    13     8.994483e+00     1.309487e-03
    14     9.150607e+00     2.644606e-03
    15     8.923354e+00     5.122766e-03
    16     8.961789e+00     1.025751e-02
    17     8.794764e+00     2.044262e-02
    18     8.813170e+00     4.089751e-02
    19     8.657163e+00     8.175219e-02
    20     8.691038e+00     1.635092e-01
    21     8.516176e+00     3.269789e-01
    22     8.56008

   248     1.628987e-01     3.954981e-07
   249     1.627257e-01     4.018505e-07
   250     1.625446e-01     3.950474e-07
   251     1.623708e-01     4.015533e-07
   252     1.621894e-01     3.946088e-07
   253     1.620148e-01     4.012647e-07
   254     1.618333e-01     3.941817e-07
   255     1.616582e-01     4.009845e-07
   256     1.614766e-01     3.937658e-07
   257     1.613011e-01     4.007125e-07
   258     1.611196e-01     3.933609e-07
   259     1.609439e-01     4.004484e-07
   260     1.607626e-01     3.929664e-07
   261     1.605867e-01     4.001919e-07
   262     1.604057e-01     3.925821e-07
   263     1.602298e-01     3.999428e-07
   264     1.600492e-01     3.922077e-07
   265     1.598733e-01     3.997008e-07
   266     1.596932e-01     3.918428e-07
   267     1.595175e-01     3.994658e-07
   268     1.593378e-01     3.914872e-07
   269     1.591623e-01     3.992375e-07
   270     1.589833e-01     3.911405e-07
   271     1.588080e-01     3.990156e-07
   272     1.586

   498     1.287068e-01     1.682947e-07
   499     1.286345e-01     1.683785e-07
   500     1.285616e-01     1.684634e-07
   501     1.284881e-01     1.685476e-07
   502     1.284142e-01     1.686322e-07
   503     1.283398e-01     1.687165e-07
   504     1.282650e-01     1.688010e-07
   505     1.281898e-01     1.688853e-07
   506     1.281144e-01     1.689698e-07
   507     1.280387e-01     1.690542e-07
   508     1.279627e-01     1.691386e-07
   509     1.278865e-01     1.692231e-07
   510     1.278102e-01     1.693076e-07
   511     1.277336e-01     1.693921e-07
   512     1.276569e-01     1.694767e-07
   513     1.275800e-01     1.695613e-07
   514     1.275031e-01     1.696459e-07
   515     1.274260e-01     1.697306e-07
   516     1.273488e-01     1.698154e-07
   517     1.272715e-01     1.699002e-07
   518     1.271941e-01     1.699851e-07
   519     1.271167e-01     1.700700e-07
   520     1.270392e-01     1.701550e-07
   521     1.269617e-01     1.702401e-07
   522     1.268

   748     1.104004e-01     1.917387e-07
   749     1.103336e-01     1.918407e-07
   750     1.102669e-01     1.919427e-07
   751     1.102002e-01     1.920447e-07
   752     1.101336e-01     1.921468e-07
   753     1.100670e-01     1.922489e-07
   754     1.100005e-01     1.923510e-07
   755     1.099340e-01     1.924532e-07
   756     1.098676e-01     1.925554e-07
   757     1.098012e-01     1.926576e-07
   758     1.097349e-01     1.927598e-07
   759     1.096686e-01     1.928621e-07
   760     1.096024e-01     1.929645e-07
   761     1.095363e-01     1.930668e-07
   762     1.094702e-01     1.931692e-07
   763     1.094042e-01     1.932717e-07
   764     1.093382e-01     1.933741e-07
   765     1.092722e-01     1.934766e-07
   766     1.092064e-01     1.935791e-07
   767     1.091405e-01     1.936817e-07
   768     1.090748e-01     1.937842e-07
   769     1.090090e-01     1.938868e-07
   770     1.089434e-01     1.939895e-07
   771     1.088778e-01     1.940921e-07
   772     1.088

   999     9.668046e-02     1.717773e-07
  1000     9.663757e-02     1.715688e-07


(SingleFieldFEFunction(), NLSolversCache())

In [10]:
# Post-processing for magnetic field and current density
Bh = norm_complex(gradient(uh));

J0    = (fsource ∘ τ); # Source current density
Jeddy = 1im ⋅ omega ⋅ (fsigma ∘ τ) ⋅ mean(uh); # Eddy current density

In [11]:
writevtk(Ω, "images/transformer_gridap/transformer2", cellfields=["Az"=>abs(uh), "imAz"=>imag(uh), "normB" => Bh, "normJ" => abs(J0 + Jeddy)])

(["images/transformer_gridap/transformer2.vtu"],)

![Result: Magnetic Flux Density](images/transformer_gridap/transformer2.png)